# Build a reduced ETC model from RECON3D

In [1]:
from pytfa.redgem.lumpgem import LumpGEM
from pytfa.io.json import load_json_model
from cobra import Reaction
from cobra.util import Zero

#recon3d = load_json_model('./../data/GEM_Recon3_thermo_genes.json')
recon3d = load_json_model('./../data/GEM_Recon3_thermo_genes_cleaned_constraints.json')

recon3d.solver.configuration.tolerances.feasibility = 1e-9
recon3d.solver.configuration.tolerances.optimality = 1e-9 

# Resting membrane potential
recon3d.compartments['c']['membranePot']['e'] = 60
recon3d.compartments['e']['membranePot']['c'] = -60
# Resting mitochondrial membrane potential
recon3d.compartments['m']['membranePot']['c'] = 150
recon3d.compartments['c']['membranePot']['m'] = -150

recon3d.compartments['i']['pH'] = 7.3 # 0.1 more than cytosol

recon3d.repair()


2025-02-28 13:19:37,594 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [2]:
# Thermodynamic annotation of Sodium and Potassium
recon3d.thermo_data['metabolites']['cpd00205']['deltaGf_std'] = 1.0
recon3d.thermo_data['metabolites']['cpd00205']['deltaGf_err'] = 0.1
recon3d.thermo_data['metabolites']['cpd00205']['error']  = 'Nil'
recon3d.thermo_data['metabolites']['cpd00205']['pKa']  = []
recon3d.thermo_data['metabolites']['cpd00971']['pKa']  = []

In [3]:
# Clean constraints and variables to rebuild thermo 

# from tqdm import tqdm
# from pytfa.optim.constraints import ReactionConstraint
# from pytfa.optim.variables import ReactionVariable
# from pytfa.optim.utils import get_all_subclasses

# all_cons_subclasses = get_all_subclasses(ReactionConstraint)
# all_var_subclasses = get_all_subclasses(ReactionVariable)


# # remove all thermo variables and constratines 
# for var_subclass in all_var_subclasses:
#     var_subclass = recon3d._var_kinds[var_subclass.__name__]
#     for var in tqdm(var_subclass):
#         recon3d.remove_variable(var)

# for cons_subclass in all_cons_subclasses:
#     cons_subclass = recon3d._cons_kinds[cons_subclass.__name__]
#     for cons in tqdm(cons_subclass):
#         recon3d.remove_constraint(cons)

# recon3d.repair()

# from pytfa.io.json import save_json_model
# save_json_model(recon3d, './../data/GEM_Recon3_thermo_genes_cleaned_constraints.json')


In [4]:
######################################################
# Curate the redox thermodynamics 
######################################################

# Make sure there is only a hydrite different between evert NADH/NAD+ and NADPH/NADP+ pair 

# Correct nadh with a H+
recon3d.thermo_data['metabolites']['cpd00004']['formula'] = 'C21H28N7O14P2'
recon3d.thermo_data['metabolites']['cpd00004']['charge_std'] = -1

# Correct nadph with a H+
recon3d.thermo_data['metabolites']['cpd00005']['formula'] = 'C21H27N7O17P3'
recon3d.thermo_data['metabolites']['cpd00005']['charge_std'] = -3

# Correct fad and fadh2
# Major form is FAD(-3) at pH 7
recon3d.thermo_data['metabolites']['cpd00015']['formula'] = 'C27H30N9O15P2' # -3 Charged form 
recon3d.thermo_data['metabolites']['cpd00015']['charge_std'] = -3

# Major form is FADH2(0) at pH 7
recon3d.thermo_data['metabolites']['C01352']['formula'] = 'C27H33N9O15P2' # -2 Charged form 
recon3d.thermo_data['metabolites']['C01352']['charge_std'] = -2


In [5]:
######################################################
# Curate the electron transport chain thermodynamics #
######################################################


# 1kcal = 4.184e3 J
F = 96485/4.184e3 # Faraday constant in kcal/V

# Cytochrome redox pair 
# cyt c (Fe3+) + e- -> cyt c (Fe2+)
dE0_cytochrome = 0.25 # V
n = 1
dG = -n*F*dE0_cytochrome

dG0_Fe2  = -18.85 # kJ/mol
dG0_Fe3  = dG0_Fe2 - dG

# Add an entry for the cytochrome c Fe2+
recon3d.compounds_data['cpd00110'] = recon3d.compounds_data['cpd00109'].copy()
recon3d.compounds_data['cpd00110']['id'] = 'cpd00110'
recon3d.compounds_data['cpd00110']['name'] = 'Cytochrome c Fe2+'
recon3d.compounds_data['cpd00110']['struct_cues'] = {'Fe2': 1}
recon3d.compounds_data['cpd00110']['deltaGf_std'] = dG0_Fe2
recon3d.compounds_data['cpd00110']['deltaGf_err'] = 0.5
recon3d.compounds_data['cpd00110']['charge_std'] = 0
recon3d.compounds_data['cpd00110']['formula'] = 'C42H54FeN8O6S2'
recon3d.compounds_data['cpd00110']['mass_std'] = 884
recon3d.compounds_data['cpd00110']['pKa'] = []

# Add an entry for the cytochrome c Fe3+
recon3d.compounds_data['cpd00109']['id'] = 'cpd00109'
recon3d.compounds_data['cpd00109']['name'] = 'Cytochrome c Fe3+'
recon3d.compounds_data['cpd00109']['struct_cues'] = {'Fe3': 1}
recon3d.compounds_data['cpd00109']['deltaGf_std'] = dG0_Fe3
recon3d.compounds_data['cpd00109']['deltaGf_err'] = 0.5
recon3d.compounds_data['cpd00109']['charge_std'] = 1
recon3d.compounds_data['cpd00109']['formula'] = 'C42H54FeN8O6S2'
recon3d.compounds_data['cpd00109']['mass_std'] = 884
recon3d.compounds_data['cpd00109']['pKa'] = []

# Flavin redox pair (is this the right pair?)
# FAD + 2H+ + 2e- -> FADH2 (currently FAD is deprotonated so missing an H+)
E0_flavin = -0.219 # V
n = 2 
dG = -n*F*E0_flavin
dG_H = recon3d.metabolites.h_m.thermo['deltaGf_std']

dG0_FAD  = -229.75 # kJ/mol
dG0_FADH2  = dG0_FAD - dG + dG_H # Account for the additional H+ in the reaction


# FAD https://modelseed.org/biochem/compounds/cpd00015
fad_data = recon3d.metabolites.fad_m.thermo.__dict__
fad_data['deltaGf_std'] = dG0_FAD
fad_data['deltaGf_err'] = 2.22
fad_data['mass_std'] = recon3d.metabolites.fad_m.thermo['mass']
recon3d.compounds_data['cpd00015'].update(fad_data) # Update the compound databse 

# FADH2 https://modelseed.org/biochem/compounds/cpd00982 # Database entry C01352
fadh2_data = recon3d.metabolites.fadh2_m.thermo.__dict__
fadh2_data['deltaGf_std'] = dG0_FADH2
fadh2_data['deltaGf_err'] = 2.22
fadh2_data['mass_std'] = recon3d.metabolites.fadh2_m.thermo['mass']
recon3d.compounds_data['C01352'].update(fadh2_data) # Update the compound databse



In [6]:
# annotate ficytC_m
recon3d.metabolites.ficytC_m.annotation['seed_id'] = 'cpd00109' # This is the Fe3+ form
recon3d.metabolites.focytC_m.annotation['seed_id'] = 'cpd00110' # This is the Fe2+ form

recon3d.metabolites.ficytC_m.charge = 1 #2
recon3d.metabolites.focytC_m.charge = 0 #3

# Model Electron transfer flavoprotein oxidized/redcued
# etfox_m as fad_m and etfrd_m as fadh2_m
recon3d.metabolites.etfox_m.annotation['seed_id'] = 'cpd00015'
recon3d.metabolites.etfrd_m.annotation['seed_id'] = 'C01352'


In [7]:
# Thermo for sodium
recon3d.compounds_data['cpd00971']['deltaGf_std'] = 1.0
recon3d.compounds_data['cpd00971']['deltaGf_err'] = 0.1
recon3d.compounds_data['cpd00971']['error']  = 'Nil'

In [8]:
###########################################
## Prepare the model lumpGEM
###########################################


# Add dummy reactions for ATP hydrolysis
cyt_atp2adp = Reaction('cyt_atp2adp')
recon3d.add_reactions([ cyt_atp2adp,])
cyt_atp2adp.reaction = 'atp_c + h2o_c --> adp_c + pi_c + h_c'

# ADD Phosphate translocase reaction (PiC)
# TODO: What is the correct stoichiometry for this reaction?
PiC = Reaction('PiC')
recon3d.add_reactions([ PiC,])
PiC.reaction = 'pi_c + h_c -> pi_m + h_m'


# This converts cobra reactions to pytfa (adds binary variables/constrains)
recon3d.objective = Zero # Symbolic zero objective

recon3d.prepare()
recon3d.convert()


2025-02-28 13:19:47,974 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...
2025-02-28 13:19:58,273 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2025-02-28 13:19:58,273 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2025-02-28 13:19:58,274 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 13:19:58,274 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 13:19:58,275 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2025-02-28 13:19:58,275 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2025-02-28 13:19:58,275 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 13:19:58,275 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 13:19:58,293 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H9

In [9]:
[(r.id, r.reaction) for r in recon3d.metabolites.mal_L_c.reactions]

[('MALOAtm', 'mal_L_m + oaa_c <=> mal_L_c + oaa_m'),
 ('CITtam', 'cit_c + mal_L_m <=> cit_m + mal_L_c'),
 ('r2391', 'HC00342_c + h_c + mal_L_m --> HC00342_m + h_m + mal_L_c'),
 ('r0913', 'icit_m + mal_L_c <=> icit_c + mal_L_m'),
 ('HMR_4964', 'cit_c + h_c + mal_L_m --> cit_m + h_m + mal_L_c'),
 ('MALSO3tm', 'mal_L_c + so3_m <=> mal_L_m + so3_c'),
 ('MDH', 'mal_L_c + nad_c <=> h_c + nadh_c + oaa_c'),
 ('FUM', 'fum_c + h2o_c <=> mal_L_c'),
 ('MALSO4tm', 'mal_L_c + so4_m <=> mal_L_m + so4_c'),
 ('ME2', 'mal_L_c + nadp_c --> co2_c + nadph_c + pyr_c'),
 ('MALtm', 'mal_L_c + pi_m <=> mal_L_m + pi_c'),
 ('r2379', 'HC00342_m + mal_L_c <=> HC00342_c + mal_L_m'),
 ('MALTSULtm', 'mal_L_c + tsul_m <=> mal_L_m + tsul_c'),
 ('MAL_Lte', 'mal_L_e <=> mal_L_c'),
 ('r0822', 'fum_c + mal_L_m <=> fum_m + mal_L_c'),
 ('MAL_Ltx', 'mal_L_x <=> mal_L_c'),
 ('AKGMALtm', 'akg_m + mal_L_c <=> akg_c + mal_L_m'),
 ('r2387', 'h_c + icit_c + mal_L_m --> h_m + icit_m + mal_L_c')]

In [10]:
# Define the core model as all reactions that connect the ETC intermediates (RedGEM, paper = D0 ) 
# Quinones, Cytochromes, and ATP synthase

ubiquinone_reqations = [r.id for r in recon3d.metabolites.q10_m.reactions if recon3d.metabolites.q10h2_m in r.metabolites]
ferrocytochrome_reactions = [r.id for r in recon3d.metabolites.focytC_m.reactions if recon3d.metabolites.ficytC_m in r.metabolites]

# Malate aspartate shuttle 
aspartate_malate_shuttle = ['MDH', 'ASPTA', 'ASPTAm', 'MDHm', 'ASPGLUm', 'AKGMALtm']

# Glycerol 3-phosphate shuttle muscle
glycerol_3_phosphate_shuttle = ['G3PD1','r0205']

# Lactate shuttle muscle
lactate_shuttle = ['LDH_L', 'LDH_Lm','L_LACtm','PYRt2m']

core_reaction_ids = list(set(ubiquinone_reqations + ferrocytochrome_reactions + ['ATPS4mi'] + glycerol_3_phosphate_shuttle + lactate_shuttle + aspartate_malate_shuttle))
core_reactions = [r for r in recon3d.reactions if r.id in core_reaction_ids]

# Exclude some reactions from core 
exclude_reactions = ['L_LACDcm',]
core_reactions = [r for r in core_reactions if r.id not in exclude_reactions]

# Print a table with the core reactions
print('Core reactions')
print('--------------')
for r in core_reactions:
    print(r.id, r.reaction, r.thermo)



Core reactions
--------------
AKGMALtm akg_m + mal_L_c <=> akg_c + mal_L_m {'isTrans': True, 'computed': True, 'deltaGR': 0.0012994733051527874, 'deltaGrxn': 0, 'deltaGRerr': 2}
ASPGLUm asp_L_m + glu_L_c + h_c --> asp_L_c + glu_L_m + h_m {'isTrans': True, 'computed': True, 'deltaGR': -4.550942133284353, 'deltaGrxn': 0, 'deltaGRerr': 2}
ASPTAm akg_m + asp_L_m <=> glu_L_m + oaa_m {'isTrans': False, 'computed': True, 'deltaGR': -1.144983579865368, 'deltaGRerr': 2}
DHORD9 dhor_S_c + q10_m --> orot_c + q10h2_m {'isTrans': False, 'computed': True, 'deltaGR': -10.478621558705754, 'deltaGRerr': 2.3526057893323307}
ETFQO etfrd_m + q10_m --> etfox_m + h_m + q10h2_m {'isTrans': False, 'computed': True, 'deltaGR': -13.419995909336194, 'deltaGRerr': 3.48528908413635}
LDH_Lm lac_L_m + nad_m <=> h_m + nadh_m + pyr_m {'isTrans': False, 'computed': True, 'deltaGR': -6.763114917731656, 'deltaGRerr': 1.6149334351607187}
L_LACtm h_c + lac_L_c --> h_m + lac_L_m {'isTrans': True, 'computed': True, 'deltaGR'

In [11]:
# Refine Glycolysis corrected delta G constraints from Lehninger:
# Units are in kJ/mol
glycolysis_thermodynamics = {
    'HEX1': -16.7,
    'PGI': 1.7,
    'PFK': -14.9,
    'FBA': 23.8,
    'TPI': 5.6, # Equilibrator value
    'GAPD': 6.3,
    'PGK': 18.8,
    'PGM': 4.4, 
    'ENO': 7.5,
    'PYK': -31.4,
    'LDH_L': 25.1,
    'LDH_Lm': 33.1,
    }

# Converstion to kcal/mol
scaling = 0.239006
for r_id, dg in glycolysis_thermodynamics.items():
    dgo = recon3d.delta_gstd.get_by_id(r_id).variable
    try:
        dgo.ub = dg * scaling + 0.5
        dgo.lb = dg * scaling - 0.5
    except ValueError:
        dgo.lb = dg * scaling - 0.5
        dgo.ub = dg * scaling + 0.5

    recon3d.optimize()
    print(f"{r_id} : {dgo.lb} < {dgo.primal} < {dgo.ub} kcal/mol")
    

sol = recon3d.optimize()

HEX1 : -4.491400199999999 < -3.4914001999999997 < -3.4914001999999997 kcal/mol
PGI : -0.09368979999999999 < -0.09368979999999999 < 0.9063102000000001 kcal/mol
PFK : -4.0611894 < -4.0611894 < -3.0611894 kcal/mol
FBA : 5.1883428 < 5.1883428 < 6.1883428 kcal/mol
TPI : 0.8384335999999999 < 1.1480295708646202 < 1.8384336 kcal/mol
GAPD : 1.0057378 < 1.0057378 < 2.0057378 kcal/mol
PGK : 3.9933128 < 3.9933128 < 4.9933128 kcal/mol
PGM : 0.5516264 < 0.5516264 < 1.5516264 kcal/mol
ENO : 1.292545 < 1.292545 < 2.292545 kcal/mol
PYK : -8.004788399999999 < -8.004788399999999 < -7.0047884 kcal/mol
LDH_L : 5.4990506 < 5.4990506 < 6.4990506 kcal/mol
LDH_Lm : 7.4110986 < 7.4110986 < 8.411098599999999 kcal/mol


In [12]:
# Refine TCA corrected delta G constraints from Lehninger:
# Units are in kJ/mol
tca_thermodynamics = {
    'PDHm' : -33.4,
    'CSm' : -32.3,
    'ACONTm' : 13.3,
    'ICDHxm' : 10.0, # Equilibrator value ?!?!
    'AKGDm' : -33.5,
    'SUCOAS1m' : -2.9,
    #'r0509' : 0, 
    'FUMm': -3.8,
    'MDHm' : 29.7,
    'MDH': 29.7 + 2.3 * 2 * 310 * 2.26 * (0.8) /1000 , # MDH delta proton correction in cytosol (nH 2.3 RT * delta pH)
    'ADK1': 0.3, # Equilibrator value ?!
    'NDPK1m': -2.7, # Equilibrator value  (EC: 2.7.4.6)
    'FADH2ETC': -70/2, # Equilibrator  value ?
    'G3PD1': 22.6, # Equilibrator value ?!?
    'r0205': -59.7, # Equilibrator value ?!?
    'PiC': 2.3 * 310 * 2.26 * (-0.8) / 1000 ,  # Charge NEUTRAL transport of phophate -> only proton force (nH 2.3 RT * delta pH)
    'PPA': -19, # 
    'FACOAL160i': -15, # Lehninge
    'BDHm': 8.0, # Equilibrator value (EC: 1.1.1.30)
    'OCOAT1m': 12.6, # Equilibrator value (EC: 2.8.3.5)
    'ACACT1rm': 25, # Equilibrator value (EC: 2.3.1.9)
    'C160CPT1': -2.2, # Equilibrator value (EC: 2.3.1.21 reverse)
    'C160CPT1': 2.2, # Equilibrator value (EC: 2.3.1.21)
}

# Converstion to kcal/mol
scaling = 0.239006
for r_id, dg in tca_thermodynamics.items():
    dgo = recon3d.delta_gstd.get_by_id(r_id).variable
    try:
        dgo.ub = dg * scaling + 0.5
        dgo.lb = dg * scaling - 0.5
    except ValueError:
        dgo.lb = dg * scaling - 0.5
        dgo.ub = dg * scaling + 0.5

    recon3d.optimize()
    print(f"{r_id} : {dgo.lb} < {dgo.primal} < {dgo.ub} kcal/mol")
    

sol = recon3d.optimize()

PDHm : -8.482800399999999 < -8.482800399999999 < -7.4828003999999995 kcal/mol
CSm : -8.2198938 < -8.2198938 < -7.2198937999999995 kcal/mol
ACONTm : 2.6787798 < 2.6787798 < 3.6787798 kcal/mol
ICDHxm : 1.89006 < 1.89006 < 2.89006 kcal/mol
AKGDm : -8.506701 < -8.506701 < -7.506701 kcal/mol
SUCOAS1m : -1.1931174 < -1.1931174 < -0.1931174 kcal/mol
FUMm : -1.4082227999999999 < -0.4082228 < -0.4082228 kcal/mol
MDHm : 6.5984782 < 6.5984782 < 7.5984782 kcal/mol
MDH : 7.214685381248 < 7.214685381248 < 8.214685381248 kcal/mol
ADK1 : -0.4282982 < -0.4282982 < 0.5717018 kcal/mol
NDPK1m : -1.1453162 < -1.1453162 < -0.1453162 kcal/mol
FADH2ETC : -8.86521 < -8.86521 < -7.865209999999999 kcal/mol
G3PD1 : 4.9015356 < 4.9015356 < 5.9015356 kcal/mol
r0205 : -14.7686582 < -14.116504745747534 < -13.7686582 kcal/mol
PiC : -0.808103590624 < -0.808103590624 < 0.191896409376 kcal/mol
PPA : -5.041114 < -5.041114 < -4.041114 kcal/mol
FACOAL160i : -4.08509 < -3.08509 < -3.08509 kcal/mol
BDHm : 1.412048 < 1.412048 

In [13]:
# Check FADH2ETC reaction
from pytfa.thermo.utils import check_reaction_balance
check_reaction_balance(recon3d.reactions.FADH2ETC)

'balanced'

In [14]:
# Save the reactions bounds from the tissue specific model
tissue_reaction_ko = dict()
for reaction in recon3d.reactions:
    if reaction.lower_bound==0 and reaction.upper_bound==0:
        tissue_reaction_ko[reaction.id] = (reaction.lower_bound, reaction.upper_bound)

In [15]:
# Sanity check
# Compute the redox potential of the Q10/Q10H2 couple in the model
# Q10 + 3H+ + 2e- -> Q10H2 
dG = recon3d.metabolites.fad_m.thermo['deltaGf_std'] + 3*recon3d.metabolites.h_m.thermo['deltaGf_std'] - recon3d.metabolites.fadh2_m.thermo['deltaGf_std']
# Convert to V
# ΔG=−nFE 
# 1kcal = 4.184e3 J
F = 96485/4.184e3
n = 2

E = dG / (n * F)
E # should be arround -0.22 V

-0.19426133595895678

In [16]:
# For future reference making debuging easier
#recon3d.solver.problem.conflict.refine()
#recon3d.solver.problem.conflict.write('bla.txt')

In [17]:
# Make sure that transporters dont have unrealistic equilibrium constants
EPSILON_DG = 1e-6

exceptions = ['ATPS4mi','CYOR_u10mi','CYOOm2i','NADH2_u10mi','PiC',]
# Print transport deltaG0
for r in recon3d.reactions:
    if r.thermo['isTrans'] and (r.id not in exceptions):
        try:
            print(f"{r.id} : {r.reaction} {r.thermo['deltaGR']} kcal/mol")
            dgo = recon3d.delta_gstd.get_by_id(r.id)
            dgo.variable.ub = r.thermo['deltaGR'] + EPSILON_DG
            dgo.variable.lb = r.thermo['deltaGR'] - EPSILON_DG
        except KeyError:
            pass


recon3d.optimize()


10FTHF5GLUtl : 10fthf5glu_c --> 10fthf5glu_l 1000.0 kcal/mol
10FTHF5GLUtm : 10fthf5glu_m --> 10fthf5glu_c 1000.0 kcal/mol
10FTHF6GLUtl : 10fthf6glu_c --> 10fthf6glu_l 1000.0 kcal/mol
10FTHF6GLUtm : 10fthf6glu_m --> 10fthf6glu_c 1000.0 kcal/mol
10FTHF7GLUtl : 10fthf7glu_c --> 10fthf7glu_l 1000.0 kcal/mol
10FTHF7GLUtm : 10fthf7glu_m --> 10fthf7glu_c 1000.0 kcal/mol
10FTHFtl : 10fthf_c <=> 10fthf_l -0.8807943970895735 kcal/mol
10FTHFtm : 10fthf_c <=> 10fthf_m 6.918311995294456 kcal/mol
11DOCRTSLtm : 11docrtsl_c <=> 11docrtsl_m 0.0 kcal/mol
11DOCRTSLtr : 11docrtsl_c <=> 11docrtsl_r 0.0 kcal/mol
11DOCRTSTRNtm : 11docrtstrn_c <=> 11docrtstrn_m 2.842170943040401e-14 kcal/mol
11DOCRTSTRNtr : 11docrtstrn_c <=> 11docrtstrn_r 0.0 kcal/mol
1MNCAMti : 1mncam_c + atp_c + h2o_c --> 1mncam_e + adp_c + h_c + pi_c 1000.0 kcal/mol
24_25DHVITD2t : 2425dhvitd2_c --> 2425dhvitd2_e 0.0 kcal/mol
24_25DHVITD2tm : 2425dhvitd2_m --> 2425dhvitd2_c 0.0 kcal/mol
24_25DHVITD3t : 2425dhvitd3_c --> 2425dhvitd3_e 0.0 k

,fluxes,reduced_costs
10FTHF5GLUtl,0.0,None
10FTHF5GLUtm,0.0,None
10FTHF6GLUtl,0.0,None
10FTHF6GLUtm,0.0,None
10FTHF7GLUtl,0.0,None
...,...,...
CYOOm2i,0.0,None
ARTPLM1,0.0,None
ARTPLM2,0.0,None
cyt_atp2adp,0.0,None


In [18]:
recon3d.reactions.get_by_id('cyt_atp2adp').bounds = (1, 200)
recon3d.reactions.get_by_id('PiC').bounds = (0, 400)

In [19]:
# Open bounds from -100 -> -200 and 100 to 200
for r in recon3d.reactions:
    if r.bounds[0] == -100:
        r.lower_bound = -200
    if r.bounds[1] == 100:
        r.upper_bound = 200 

In [20]:
# By default only allow secretions for lumping + defined medium
for rxn in recon3d.boundary:
    rxn.bounds = (0, 200)

# Allow for protons exchange and oxygen uptake
recon3d.reactions.get_by_id('EX_h_e').bounds = (-200.0, 200.0)
recon3d.reactions.get_by_id('EX_h2o_e').bounds = (-200.0, 200.0)
recon3d.reactions.get_by_id('EX_o2_e').bounds = (-200.0, 0.0)

# Lactate and pyruvate transporters should be reversible
recon3d.reactions.get_by_id('L_LACtm').bounds = (-100, 100)
recon3d.reactions.get_by_id('PYRt2m').bounds = (-100, 100)


# Test the model (For lumpting close all carbon sources)
recon3d.reactions.get_by_id('EX_lac_L_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_glc_D_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_bhb_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_hdca_e').bounds = (-1, 100)

recon3d.reactions.get_by_id('ATPS4mi').bounds = (0,200)
recon3d.reactions.get_by_id('ATPtm').bounds = (-200,200)
# These are some FA reaction that can cause weird cycling
recon3d.reactions.r0310.bounds = (0, 0)
recon3d.reactions.HMR_3121.bounds = (0, 0)

# This can bypass the Malate shuttle should not
recon3d.reactions.MALOAtm.bounds = (0, 0)   

# Remove the rxn with the weired stoich 
recon3d.reactions.CYOOm3i.bounds = (0, 0)

# Block the ETC LDH (very little evidence for this in general)
recon3d.reactions.L_LACDcm = (0,0)

# Allow this reaction to be reversible (in reality this runs backwards)
# See Lehninger Principles of Biochemistry
recon3d.reactions.ACACT1rm.bounds = (-200,200)

# Block the force reaction to bo forward (this should not make ATP)
# https://www.proteinatlas.org/ENSG00000106992-AK1/tissue+cell+type
recon3d.reactions.ADK1.bounds = (0,100) # THIS IS THE MAIN ADK expressed in muscle (AK1 and AK2) 

# This is an entorcyte specific transport not active in muscle
recon3d.reactions.PALFATPtc.bounds = (0,0)

# Block nadph reactions (erros in beta ox)
for r in recon3d.metabolites.nadp_m.reactions:
    r.bounds = (0,0)
for r in recon3d.metabolites.nadp_c.reactions:
    r.bounds = (0,0)

# Test w/o direct ATP production in SUCOASm
recon3d.reactions.SUCOASm.bounds = (0,0)

# Relax dGo of NADH2_u10mi 
# wrong calculation since not 6 but only 4 protons are transported across the membrane
# the other ones are stashed onto qh2 
dg = -3.957108259354982 + (6.544013627216948 + 24.905879999999996) * 4/6 

# From this publication 
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2931722/
# The standard Gibbs free energy change for the reaction is -111 kJ/mol
dg =  -111* 0.24 + (6.544013627216948 + 24.905879999999996) * 4/6 

recon3d.delta_gstd.NADH2_u10mi.variable.lb = dg - 5
recon3d.delta_gstd.NADH2_u10mi.variable.ub = dg + 5


recon3d.objective = recon3d.reactions.get_by_id('cyt_atp2adp')

sol = recon3d.optimize()
print(sol)


<Solution 106.000 at 0x7fa7021ec670>


In [21]:
import numpy as np
from pytfa.optim.constraints import ModelConstraint

# Force ATP/ADP ratio to be 10
# Cytosolic ATP/ADP ratio
# https://pubmed.ncbi.nlm.nih.gov/749453/



expression = recon3d.log_concentration.atp_c.variable - recon3d.log_concentration.adp_c.variable
id_='atp_c_adp_c_ratio'

recon3d.add_constraint(ModelConstraint,
                        hook=recon3d,
                        expr=expression,
                        id_=id_, 
                        lb=np.log(10), 
                        ub=np.log(100),
                        )
print(f"Integrate cytosolic ATP/ADP ratio: 10-50 ")
sol = recon3d.optimize()
print(f"ATP yield: {sol.objective_value}")


# Force ATP/ADP ratio to bearround 2 and 8
# Cytosolic ATP/ADP ratio
# https://www.mdpi.com/1422-0067/23/10/5550


expression = recon3d.log_concentration.atp_m.variable - recon3d.log_concentration.adp_m.variable
id_='atp_m_adp_m_ratio'

recon3d.add_constraint(ModelConstraint,
                        hook=recon3d,
                        expr=expression,
                        id_=id_, 
                        lb=np.log(2), 
                        ub=np.log(8),
                        )
print(f"Integrate mitochondrial ATP/ADP ratio: 2-8")
sol = recon3d.optimize()
sol = recon3d.optimize()
print(f"ATP yield: {sol.objective_value}")

# mitochondrial GTP/GDP ratio
expression = recon3d.log_concentration.gtp_m.variable - recon3d.log_concentration.gdp_m.variable

recon3d.add_constraint(ModelConstraint,
                        hook=recon3d,
                        expr=expression,
                        id_='gtp_m_gdp_m_ratio', 
                        lb=np.log(10), 
                        ub=np.log(300),
                        )
print(f"Integrate mitochondrial GTP/GDP ratio: 10-300")
recon3d.optimize()
sol = recon3d.optimize()
print(f"ATP yield: {sol.objective_value}")        

# mitochondrial NAD/NADH ratio
expression = recon3d.log_concentration.nad_m.variable - recon3d.log_concentration.nadh_m.variable

recon3d.add_constraint(ModelConstraint,
                        hook=recon3d,
                        expr=expression,
                        id_='nad_m_nadh_m_ratio', 
                        lb=np.log(6), 
                        ub=np.log(8),
                        )
print(f"Integrate mitochondrial NAD+/NADH ratio: 6-8")
recon3d.optimize()
sol = recon3d.optimize()
print(f"ATP yield: {sol.objective_value}")  

# NOTE: This is currently not recomended gives stupid networks because of wrong deltag G values in glycolyis and TCA
# TODO: Add some manual curation of the dG values for the reactions in the recon3d model then rerun this

# # Cytoplasmic NAD/NADH ratio
expression = recon3d.log_concentration.nad_c.variable - recon3d.log_concentration.nadh_c.variable

recon3d.add_constraint(ModelConstraint,
                        hook=recon3d,
                        expr=expression,
                        id_='nad_c_nadh_c_ratio', 
                        lb=np.log(60), 
                        ub=np.log(700),
                        )

print(f"Integrate cytoplasmic NAD+/NADH ratio: 60-700")
recon3d.optimize()
sol = recon3d.optimize()
print(f"ATP yield: {sol.objective_value}")  

Integrate cytosolic ATP/ADP ratio: 10-50 
ATP yield: 106.0
Integrate mitochondrial ATP/ADP ratio: 2-8
ATP yield: 106.0
Integrate mitochondrial GTP/GDP ratio: 10-300
ATP yield: 106.0
Integrate mitochondrial NAD+/NADH ratio: 6-8
ATP yield: 106.0
Integrate cytoplasmic NAD+/NADH ratio: 60-700
ATP yield: 106.0


In [22]:
# Print non-zero fluxes that are not transport reactions
for r in recon3d.reactions:
    if abs(sol.fluxes[r.id]) > 1e-6  and recon3d.metabolites.get_by_id('nadh_m') in r.metabolites:
        print(r.id, sol.fluxes[r.id], r.reaction, r.thermo)


AKGDm 8.0 akg_m + coa_m + nad_m --> co2_m + nadh_m + succoa_m {'isTrans': False, 'computed': True, 'deltaGR': -20.408539828521327, 'deltaGRerr': 1.9903323843016776}
FAOXC160 1.0 7.0 coa_m + 7.0 fad_m + 7.0 h2o_m + 7.0 nad_m + pmtcoa_m --> 8.0 accoa_m + 7.0 fadh2_m + 7.0 nadh_m {'isTrans': False, 'computed': True, 'deltaGR': -41.384975377771994, 'deltaGRerr': 19.0660846174562}
ICDHxm 8.0 icit_m + nad_m --> akg_m + co2_m + nadh_m {'isTrans': False, 'computed': True, 'deltaGR': -7.476665493632595, 'deltaGRerr': 1.7170517872213407}
MDHm 8.0 mal_L_m + nad_m <=> h_m + nadh_m + oaa_m {'isTrans': False, 'computed': True, 'deltaGR': -6.762883221652373, 'deltaGRerr': 1.6149334351607187}
NADH2_u10mi 31.0 6.0 h_m + nadh_m + q10_m --> h_c + 4.0 h_i + nad_m + q10h2_m {'isTrans': True, 'computed': True, 'deltaGR': 22.84204887623445, 'deltaGrxn': -4.456864750982504, 'deltaGRerr': 3.07450548869245}


In [23]:
# Print non-zero fluxes that are not transport reactions
for r in recon3d.reactions:
    if  recon3d.metabolites.get_by_id('h_c') in r.metabolites and sol.fluxes[r.id] != 0:
        print(r.id, sol.fluxes[r.id], r.reaction, r.check_mass_balance())

FACOAL160i 1.0 atp_c + coa_c + hdca_c --> amp_c + h_c + pmtcoa_c + ppi_c {}
r1155 15.5 2obut_c + h_c --> 2obut_m + h_m {}
r1454 15.5 2obut_m + h_c --> 2obut_c + h_m {}
The 1.0 h_e <=> h_c {}
ADK1 1.0 amp_c + atp_c + h_c --> 2.0 adp_c {}
PPA 1.0 h2o_c + ppi_c --> h_c + 2.0 pi_c {}
NADH2_u10mi 31.0 6.0 h_m + nadh_m + q10_m --> h_c + 4.0 h_i + nad_m + q10h2_m {'charge': 1.0}
cyt_atp2adp 106.0 atp_c + h2o_c --> adp_c + h_c + pi_c {}
PiC 108.0 h_c + pi_c --> h_m + pi_m {}


In [24]:
# recon3d.reactions.get_by_id('ATPS4mi').bounds = (0,200)
# recon3d.objective = recon3d.reactions.get_by_id('EX_lac_L_e')
# sol = recon3d.optimize()
# print(sol)
# # Constraint max lactate secretion
# # recon3d.reactions.get_by_id('EX_lac_L_e').bounds = (sol.objective_value, 100)

In [25]:
from pytfa.io.json import save_json_model

save_json_model(recon3d, './../data/GEM_Recon3_Thermo_Lehninger_Curated.json')

In [26]:
# Test the model (For lumpting close all carbon sources)
recon3d.reactions.get_by_id('EX_lac_L_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_glc_D_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_bhb_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_hdca_e').bounds = (0, 100)



In [27]:
# Generate a lump for each carbon source 
EPSILON = 1e-6

# LumpGEM parameters
params = {
    'core_subsystems': [],
    'extracellular_system': [],
    'timeout': 3600,  # max time in s
    "constraint_method": 'both',
    # Stuff we dont need for this purpose
    "small_metabolites": [],
    "cofactor_pairs": [],
    "inorganics": [],
    "max_lumps_per_BBB": 10  # Maximal number of alternatives
}

In [28]:
def find_lumps(carbon_source, n_carbon, model, params=params, 
                core_reactions=[], method='min',
                output='EX_co2_e', tissue_reaction_ko=None,
                max_energy_yield=0.9):  
        # Lump the reactions 
        
        params['growth_rate'] = n_carbon-EPSILON
        params['biomass_rxns'] =  [output,]

        resp_model = model.copy()

        # If tissue bounds are provided
        if not tissue_reaction_ko is None:
            for r_id, bounds in tissue_reaction_ko.items():
                resp_model.reactions.get_by_id(r_id).bounds = bounds
    
        resp_model.reactions.get_by_id(carbon_source).bounds = (-1-EPSILON, -1+EPSILON)
        resp_model.objective = resp_model.reactions.get_by_id(output)
        resp_model.objective_direction = 'max'
        sol = resp_model.optimize()
        print("Max CO2 production: ", sol.objective_value)

        # Maximal ATP production 
        resp_model.reactions.get_by_id(output).bounds = (n_carbon-EPSILON, n_carbon+EPSILON)
        resp_model.objective = resp_model.reactions.get_by_id('cyt_atp2adp')
        resp_model.objective_direction = 'max'
        sol = resp_model.optimize()
        print("Max ATP production: ", sol.objective_value)

        # Constraint max ATP production
        resp_model.reactions.get_by_id('cyt_atp2adp').bounds = (sol.objective_value*max_energy_yield-EPSILON*n_carbon, 200)

        # Reset CO2 constraint
        resp_model.reactions.get_by_id(output).bounds = (0, 100)

        resp_model.objective = Zero
        sol = resp_model.optimize()

        print("Test feasability: ", sol.objective_value)
        
        #LumpGEM needs to take as list of core reaction id as input
        subnetwork_extraction = LumpGEM(resp_model, core_reactions, params, min_transport=True, bigM=200 )
        lumps = subnetwork_extraction.compute_lumps(force_solve=False, method=method)

        return lumps

In [29]:
# Find the lumps for each carbon source to CO2
lumps = {}

In [30]:
carbon_source = "EX_hdca_e"
n_carbon = 16

# Force ATPs to be active for the respiratory lumps 
recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)

lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d, method='min',
                                    core_reactions=core_reaction_ids,
                                    tissue_reaction_ko=tissue_reaction_ko)

2025-02-28 13:23:07,476 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  16.000016000000016
Max ATP production:  106.000106
Test feasability:  0.0
Timeout limit is 3600s


2025-02-28 13:24:57,450 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2025-02-28 13:25:07,669 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2025-02-28 13:25:07,669 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2025-02-28 13:25:07,669 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 13:25:07,670 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 13:25:07,671 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2025-02-28 13:25:07,671 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2025-02-28 13:25:07,672 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 13:25:07,672 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 13:25:07,685 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2025-02-28 13:25:07,685 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=co2_e: 100%|██████████| 1/1 [36:44<00:00, 2204.98s/it]


In [31]:
lumps

{'EX_hdca_e': {<Metabolite co2_e at 0x7fa6234701f0>: [Lump(id_='LUMP_EX_co2_e_co2_e', metabolites=defaultdict(<class 'int'>, {'nad_m': -1.4375, 'nadh_m': 1.4375, 'adp_m': 5.587505336, 'atp_m': -5.587505336, 'h2o_m': -8.462505336, 'h_m': 6.587505336, 'oaa_m': -0.5, 'fad_m': -0.4375, 'fadh2_m': 0.4375, 'fum_m': -0.5, 'mal_L_m': 0.5, 'hdca_e': -0.0625, 'o2_m': 1.4375, 'pi_m': 5.587505336, 'succ_m': 0.5, 'h_e': -0.0625, 'co2_e': 1.0, 'h2o_e': 1.0, 'o2_e': -1.4375}), subnetwork={'ACONTm': 0.49999999999999994, 'AKGDm': 0.49999999999999994, 'ATPtm': 6.087505335529238, 'C160CPT1': 0.06249999999999999, 'C160CPT2': 0.06249999999999999, 'CO2tm': -0.9999999999999999, 'CSm': 0.49999999999999994, 'FACOAL160i': 0.06249999999999999, 'FAOXC160': 0.06249999999999999, 'FUMm': 1.275004944513589, 'H2Otm': -6.250000391015649, 'HDCAtr': 0.06249999999999999, 'ICDHxm': 0.49999999999999994, 'NDPK1m': -0.49999999999999994, 'O2tm': 1.4374999999999996, 'SUCOAS1m': -0.49999999999999994, 'r0822': 0.7750049445135889,

In [32]:
# Compute a lump for glucose respiration
carbon_source = 'EX_glc_D_e'
n_carbon = 6

recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)

# Add glycolysis lumps
lumps[carbon_source+'_ox'] = find_lumps(carbon_source, n_carbon, recon3d, method='min',
                                   core_reactions=core_reaction_ids,
                                    tissue_reaction_ko=tissue_reaction_ko)

2025-02-28 14:02:39,023 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  6.000005999999985
Max ATP production:  32.00003200000011
Test feasability:  0.0
Timeout limit is 3600s


2025-02-28 14:04:18,563 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2025-02-28 14:04:28,241 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2025-02-28 14:04:28,242 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2025-02-28 14:04:28,242 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 14:04:28,242 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 14:04:28,243 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2025-02-28 14:04:28,244 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2025-02-28 14:04:28,244 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 14:04:28,245 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 14:04:28,257 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2025-02-28 14:04:28,258 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=co2_e: 100%|██████████| 1/1 [11:23<00:00, 683.51s/it]


In [33]:
lumps

{'EX_hdca_e': {<Metabolite co2_e at 0x7fa6234701f0>: [Lump(id_='LUMP_EX_co2_e_co2_e', metabolites=defaultdict(<class 'int'>, {'nad_m': -1.4375, 'nadh_m': 1.4375, 'adp_m': 5.587505336, 'atp_m': -5.587505336, 'h2o_m': -8.462505336, 'h_m': 6.587505336, 'oaa_m': -0.5, 'fad_m': -0.4375, 'fadh2_m': 0.4375, 'fum_m': -0.5, 'mal_L_m': 0.5, 'hdca_e': -0.0625, 'o2_m': 1.4375, 'pi_m': 5.587505336, 'succ_m': 0.5, 'h_e': -0.0625, 'co2_e': 1.0, 'h2o_e': 1.0, 'o2_e': -1.4375}), subnetwork={'ACONTm': 0.49999999999999994, 'AKGDm': 0.49999999999999994, 'ATPtm': 6.087505335529238, 'C160CPT1': 0.06249999999999999, 'C160CPT2': 0.06249999999999999, 'CO2tm': -0.9999999999999999, 'CSm': 0.49999999999999994, 'FACOAL160i': 0.06249999999999999, 'FAOXC160': 0.06249999999999999, 'FUMm': 1.275004944513589, 'H2Otm': -6.250000391015649, 'HDCAtr': 0.06249999999999999, 'ICDHxm': 0.49999999999999994, 'NDPK1m': -0.49999999999999994, 'O2tm': 1.4374999999999996, 'SUCOAS1m': -0.49999999999999994, 'r0822': 0.7750049445135889,

In [34]:
# Compute a lump for glycolysis (Direct respiration makes no sense ... )
carbon_source = 'EX_glc_D_e'
n_carbon = 2
output = 'EX_lac_L_e'

# No need
recon3d.reactions.get_by_id('ATPS4mi').bounds = (0,200)

# Add glycolysis lumps
lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d, method='min',
                                   core_reactions=core_reaction_ids, output=output,
                                    tissue_reaction_ko=tissue_reaction_ko)


2025-02-28 14:16:38,120 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  2.000002
Max ATP production:  2.000001
Test feasability:  0.0
Timeout limit is 3600s


2025-02-28 14:17:57,110 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2025-02-28 14:18:06,754 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2025-02-28 14:18:06,754 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2025-02-28 14:18:06,755 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 14:18:06,755 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 14:18:06,756 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2025-02-28 14:18:06,756 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2025-02-28 14:18:06,756 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 14:18:06,757 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 14:18:06,768 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2025-02-28 14:18:06,768 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=lac_L_e: 100%|██████████| 1/1 [01:34<00:00, 94.03s/it]


In [35]:
carbon_source = "EX_lac_L_e"
n_carbon = 3
output = 'EX_co2_e'

# Force ATPs to be active for the respiratory lumps
recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)

lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d, method='min',
                                    core_reactions=core_reaction_ids,
                                    tissue_reaction_ko=tissue_reaction_ko)

2025-02-28 14:20:25,120 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  3.0000029999999995
Max ATP production:  15.000014999999962
Test feasability:  0.0
Timeout limit is 3600s


2025-02-28 14:21:44,406 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2025-02-28 14:21:54,009 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2025-02-28 14:21:54,009 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2025-02-28 14:21:54,009 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 14:21:54,010 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 14:21:54,010 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2025-02-28 14:21:54,011 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2025-02-28 14:21:54,011 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 14:21:54,011 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 14:21:54,022 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2025-02-28 14:21:54,023 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=co2_e: 100%|██████████| 1/1 [32:17<00:00, 1937.91s/it]


In [36]:
lumps

{'EX_hdca_e': {<Metabolite co2_e at 0x7fa6234701f0>: [Lump(id_='LUMP_EX_co2_e_co2_e', metabolites=defaultdict(<class 'int'>, {'nad_m': -1.4375, 'nadh_m': 1.4375, 'adp_m': 5.587505336, 'atp_m': -5.587505336, 'h2o_m': -8.462505336, 'h_m': 6.587505336, 'oaa_m': -0.5, 'fad_m': -0.4375, 'fadh2_m': 0.4375, 'fum_m': -0.5, 'mal_L_m': 0.5, 'hdca_e': -0.0625, 'o2_m': 1.4375, 'pi_m': 5.587505336, 'succ_m': 0.5, 'h_e': -0.0625, 'co2_e': 1.0, 'h2o_e': 1.0, 'o2_e': -1.4375}), subnetwork={'ACONTm': 0.49999999999999994, 'AKGDm': 0.49999999999999994, 'ATPtm': 6.087505335529238, 'C160CPT1': 0.06249999999999999, 'C160CPT2': 0.06249999999999999, 'CO2tm': -0.9999999999999999, 'CSm': 0.49999999999999994, 'FACOAL160i': 0.06249999999999999, 'FAOXC160': 0.06249999999999999, 'FUMm': 1.275004944513589, 'H2Otm': -6.250000391015649, 'HDCAtr': 0.06249999999999999, 'ICDHxm': 0.49999999999999994, 'NDPK1m': -0.49999999999999994, 'O2tm': 1.4374999999999996, 'SUCOAS1m': -0.49999999999999994, 'r0822': 0.7750049445135889,

In [37]:
carbon_source = "EX_bhb_e"
n_carbon = 4

# Force ATPs to be active for the respiratory lumps 
recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)

lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d, method='min',
                                    core_reactions=core_reaction_ids,
                                    tissue_reaction_ko=tissue_reaction_ko)

2025-02-28 14:54:58,046 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  4.000004
Max ATP production:  21.5000215
Test feasability:  0.0
Timeout limit is 3600s


2025-02-28 14:56:30,595 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2025-02-28 14:56:40,178 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2025-02-28 14:56:40,178 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2025-02-28 14:56:40,179 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 14:56:40,179 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2025-02-28 14:56:40,180 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2025-02-28 14:56:40,180 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2025-02-28 14:56:40,181 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 14:56:40,181 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2025-02-28 14:56:40,193 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2025-02-28 14:56:40,193 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=co2_e: 100%|██████████| 1/1 [03:59<00:00, 239.80s/it]


In [38]:
# Save the lumps in a json file Lumps are NamedTuple objects
import json

lumps_json = {k: [i for m,l in v.items() for i in l ] for k,v in lumps.items()}
with open('lumps.json', 'w') as f:
    json.dump(lumps_json, f, indent=4)


In [39]:
lumps_json

{'EX_hdca_e': [Lump(id_='LUMP_EX_co2_e_co2_e', metabolites=defaultdict(<class 'int'>, {'nad_m': -1.4375, 'nadh_m': 1.4375, 'adp_m': 5.587505336, 'atp_m': -5.587505336, 'h2o_m': -8.462505336, 'h_m': 6.587505336, 'oaa_m': -0.5, 'fad_m': -0.4375, 'fadh2_m': 0.4375, 'fum_m': -0.5, 'mal_L_m': 0.5, 'hdca_e': -0.0625, 'o2_m': 1.4375, 'pi_m': 5.587505336, 'succ_m': 0.5, 'h_e': -0.0625, 'co2_e': 1.0, 'h2o_e': 1.0, 'o2_e': -1.4375}), subnetwork={'ACONTm': 0.49999999999999994, 'AKGDm': 0.49999999999999994, 'ATPtm': 6.087505335529238, 'C160CPT1': 0.06249999999999999, 'C160CPT2': 0.06249999999999999, 'CO2tm': -0.9999999999999999, 'CSm': 0.49999999999999994, 'FACOAL160i': 0.06249999999999999, 'FAOXC160': 0.06249999999999999, 'FUMm': 1.275004944513589, 'H2Otm': -6.250000391015649, 'HDCAtr': 0.06249999999999999, 'ICDHxm': 0.49999999999999994, 'NDPK1m': -0.49999999999999994, 'O2tm': 1.4374999999999996, 'SUCOAS1m': -0.49999999999999994, 'r0822': 0.7750049445135889, 'r2435': 0.06249999999999999, 'The': 0

In [40]:
# Make a new model with the lumps only containing the core reactions and the lumps
reduced_model = recon3d.copy()

2025-02-28 15:01:28,588 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [41]:
boundary = [r.id for r in reduced_model.reactions if r in reduced_model.boundary]
lump_subnet_reactions = list({r for lumps in lumps_json.values() for l in lumps for r in l.subnetwork})
reduced_reactions = core_reaction_ids + lump_subnet_reactions + boundary

reactions_to_remove = [r for r in reduced_model.reactions if r.id not in reduced_reactions]
reduced_model.remove_reactions(reactions_to_remove)



In [42]:
reduced_model.prepare()
reduced_model.convert()


2025-02-28 17:42:37,613 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...
2025-02-28 18:07:29,107 - thermomodel_Recon3thermoCurated - INFO - # Model preparation done.
2025-02-28 18:07:29,108 - thermomodel_Recon3thermoCurated - INFO - # Model conversion starting...
2025-02-28 18:24:17,723 - thermomodel_Recon3thermoCurated - INFO - # Model conversion done.
2025-02-28 18:24:17,723 - thermomodel_Recon3thermoCurated - INFO - # Updating cobra_model variables...
2025-02-28 18:24:17,790 - thermomodel_Recon3thermoCurated - INFO - # cobra_model variables are up-to-date


In [43]:
# Make flux variability and remove reactions that are not active
reduced_model.objective = reduced_model.reactions.get_by_id('cyt_atp2adp') 

carbon_sources = ['EX_glc_D_e', 'EX_lac_L_e', 'EX_bhb_e', 'EX_hdca_e']

for carbon_source in carbon_sources:
    reduced_model.reactions.get_by_id(carbon_source).bounds = (-1, 0)


reduced_model.optimize()

,fluxes,reduced_costs
ACACT1rm,-1.0,None
ACONTm,13.0,None
AKGDm,13.0,None
AKGMALtm,53.0,None
ASPGLUm,53.0,None
...,...,...
NADH2_u10mi,53.0,None
CYOOm3i,0.0,None
CYOOm2i,36.5,None
cyt_atp2adp,174.5,None


In [44]:
reduced_model.medium

{'EX_bhb_e': 1,
 'EX_hdca_e': 1,
 'EX_h_e': 200.0,
 'EX_h2o_e': 200.0,
 'EX_lac_L_e': 1,
 'EX_o2_e': 200.0,
 'EX_glc_D_e': 1}

In [45]:
# TVA to remove reactions that are not active
from pytfa.analysis import variability_analysis

FVA = variability_analysis(reduced_model, kind='reactions')


2025-02-28 18:24:18,036 - thermomodel_Recon3thermoCurated - INFO - Beginning variability analysis for variable of type reactions
maximizing: 100%|██████████| 1888/1888 [1:29:37<00:00,  2.85s/it]   


In [46]:
EPSILON = 1e-9

reactions_to_remove = [r for r in reduced_model.reactions if FVA.loc[r.id, 'minimum'] >= -EPSILON and FVA.loc[r.id, 'maximum'] <= EPSILON ]
len(reactions_to_remove)

1814

In [47]:
# Load model for sake of time
# from pytfa.io.json import load_json_model
# reduced_model = load_json_model('reduced_model_no_core_20240412-090529.json')


EPSILON = 1e-9
# remove reactions that are not active min and max are zero
reactions_to_remove = [r for r in reduced_model.reactions if FVA.loc[r.id, 'minimum'] >= -EPSILON and FVA.loc[r.id, 'maximum'] <= EPSILON ]
reduced_model.remove_reactions(reactions_to_remove)
reduced_model.repair()
reduced_model.optimize()

metabolites_to_remove = [m for m in reduced_model.metabolites if len(m.reactions) == 0]
reduced_model.remove_metabolites(metabolites_to_remove)


In [48]:
reduced_model.repair()
reduced_model.optimize()

,fluxes,reduced_costs
ACACT1rm,-1.0,None
ACONTm,13.0,None
AKGDm,13.0,None
AKGMALtm,53.0,None
ASPGLUm,53.0,None
...,...,...
CYOR_u10mi,73.0,None
NADH2_u10mi,53.0,None
CYOOm2i,36.5,None
cyt_atp2adp,174.5,None


In [49]:
from pytfa.analysis import variability_analysis
FVA = variability_analysis(reduced_model, kind='reactions')
# Show the remaning reactions

2025-02-28 21:31:22,713 - thermomodel_Recon3thermoCurated - INFO - Beginning variability analysis for variable of type reactions
maximizing: 100%|██████████| 74/74 [00:00<00:00, 381.87it/s]


In [50]:
# Concentration ranges
from pytfa.optim.variables import LogConcentration
TVA = variability_analysis(reduced_model, kind=LogConcentration)

2025-02-28 21:31:23,122 - thermomodel_Recon3thermoCurated - INFO - Beginning variability analysis for variable of type <class 'pytfa.optim.variables.LogConcentration'>
maximizing: 100%|██████████| 82/82 [00:00<00:00, 124.85it/s]


In [51]:
from pytfa.io.json import save_json_model
import datetime 
# Save the reduced model as json
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
reduced_model.id = 'reduced_model_ETC_core_{}'.format(timestamp)

save_json_model(reduced_model, 'reduced_model_ETC_core_{}.json'.format(timestamp))


In [52]:
from cobra.io.json import save_json_model as cobra_save_json_model

cobra_save_json_model(reduced_model, 'reduced_model_ETC_core_fba_only_{}.json'.format(timestamp))

In [53]:
# from cobra.io.json import save_json_model as cobra_save_json_model

# cobra_save_json_model(recon3d, 'recon3d_fba_only.json')

In [54]:
# Which lumos contains r0822
for k, v in lumps.items():
    for l_id, l in v.items():
        if 'r0822' in [r for tl in l for r in tl.subnetwork]:
            print(k, l_id)


EX_hdca_e co2_e


In [55]:
reduced_model.thermo_data['metabolites'][reduced_model.metabolites.fad_m.thermo['id']]

{'id': 'cpd00015',
 'mass_std': 782,
 'name': 'Flavin Adenine Dinucleotide Oxidized',
 'formula': 'C27H30N9O15P2',
 'charge_std': -3,
 'struct_cues': {'WCH3': 2,
  'BWWCdblW': 2,
  'BWCHdblW': 2,
  'BTWWCdblW': 2,
  'RWWNW': 2,
  'WCH2W': 3,
  'WWCHW': 3,
  'PrimOH': 5,
  'WPO4nW': 1,
  'mid_phos': 1,
  'RWCHWW': 4,
  'RWOW': 1,
  'RWCHdblW': 2,
  'RWdblNW': 5,
  'TWWCdblW': 4,
  'RWCdblWW': 1,
  'WNH2': 1,
  'RWWCdblO': 2,
  'NoGroup': 1,
  'Origin': 1,
  'OCCN': 1,
  'OCNC': 1,
  'NCCN': 1,
  'CCNC': 1,
  'HeteroAromatic': 2},
 'nH_std': 30,
 'deltaGf_std': -229.75,
 'deltaGf_err': 2.22,
 'error': 'Nil',
 'pKa': [2.09, 2.3, 6.01],
 'mass': 782,
 'deltaGf_tr': 10000000,
 'pH': 8.0,
 'ionicStr': 0.15}

In [56]:
reduced_model.reactions.L_LACDcm

Reaction identifier,L_LACDcm
Name,"L-Lactate Dehydrogenase, Cytosolic/Mitochondrial"
Memory address,0x07fa69f286040
Stoichiometry,2.0 ficytC_m + lac_L_c --> 2.0 focytC_m + 2.0 h_c + pyr_c 2.0 Ferricytochrome C + _S-Lactate --> 2.0 Ferrocytochrome C + 2.0 Proton + Pyruvate
GPR,124637.1
Lower bound,0.0
Upper bound,200.0


: 